In [15]:
import os
import numpy as np
import pandas as pd

In [16]:
SCRIPT_DIR = os.getcwd()
DATA_DIR = os.path.join(SCRIPT_DIR, '..', 'data')
print DATA_DIR

/Users/aravind/movie-ranking-recommendation-system/notebooks/../data


In [17]:
meta_data = pd.read_csv(DATA_DIR + '/movies_metadata.csv')
nsfw_data = pd.read_csv(DATA_DIR + '/nsfw_score.csv')
print meta_data.shape
print nsfw_data.shape

(45466, 24)
(45047, 2)


In [18]:
#Changing the column name
nsfw_data.columns = ['id', 'nsfw_score']

#Only using relevant data from the main csv
meta_data_subset = meta_data[['budget', 'id', 'original_language', 'popularity', 'revenue']]

#Converting both fields to the same data type
meta_data_subset['id'] = meta_data_subset['id'].astype(str)
nsfw_data['id'] = nsfw_data['id'].astype(str)

In [65]:
#merging the two dataframes
df = meta_data_subset.merge(nsfw_data, on='id', how='left')